#Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools, which pymc3 provides a wrapper `model.profile` which returns a `ProfileStats` object. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [1]:
import numpy as np
from pymc3 import *
from pymc3.distributions.timeseries import *

n = 400
returns = np.genfromtxt(get_data_file('pymc3.examples', "data/SP500.csv"))[-n:]

model = Model()
with model:
    sigma, log_sigma = model.TransformedVar(
        'sigma', Exponential.dist(1. / .02, testval=.1),
        logtransform)

    nu = Exponential('nu', 1. / 10)

    s = GaussianRandomWalk('s', sigma ** -2, shape=n)

    r = T('r', nu, lam=exp(-2 * s), observed=returns)
    


:0: FutureWarning: IPython widgets are experimental and may change in the future.


Then call profile and summarize it.

In [2]:
model.profile(model.logpt).summary()


Function profiling
  Message: /home/john/Documents/workspace/pymc3/pymc3/model.py:194
  Time in 1000 calls to Function.__call__: 1.506145e-01s
  Time in Function.fn.__call__: 1.124353e-01s (74.651%)
  Time in thunks: 9.804273e-02s (65.095%)
  Total compile time: 5.747011e-01s
    Number of Apply nodes: 18
    Theano Optimizer time: 5.239811e-01s
       Theano validate time: 5.190372e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 2.100492e-02s
       Import time 2.770901e-03s

Time in all call to theano.grad() 0.000000e+00s
Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  93.7%    93.7%       0.092s       8.35e-06s     C    11000      11   theano.tensor.elemwise.Elemwise
   2.3%    96.0%       0.002s       1.12e-06s     C     2000       2   theano.tensor.elemwise.Sum
   2.1%    98.1%       0.002s       6.81e-07s     C     3000       3   theano.tensor.elemwise.DimShuffle
   1.9%   100.0%       0.002s       9.50e-07s

In [3]:
model.profile(gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /home/john/Documents/workspace/pymc3/pymc3/model.py:194
  Time in 1000 calls to Function.__call__: 2.530243e-01s
  Time in Function.fn.__call__: 2.023389e-01s (79.968%)
  Time in thunks: 1.640260e-01s (64.826%)
  Total compile time: 1.579683e+00s
    Number of Apply nodes: 45
    Theano Optimizer time: 1.478235e+00s
       Theano validate time: 2.640319e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 4.506111e-02s
       Import time 5.736113e-03s

Time in all call to theano.grad() 8.991649e-01s
Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  76.3%    76.3%       0.125s       5.21e-06s     C    24000      24   theano.tensor.elemwise.Elemwise
   7.1%    83.4%       0.012s       5.83e-06s     C     2000       2   theano.tensor.subtensor.IncSubtensor
   5.4%    88.8%       0.009s       1.28e-06s     C     7000       7   theano.tensor.elemwise.Sum
   3.6%    92.4%       0.006s       5.86e-